In [122]:
import random

class Kuhn_Poker:
    def __init__(self, p1 = 2, p2 = 2):
        self.pot = 0
        self.p1 = p1
        self.p2 = p2
        self.a1 = None
        self.a2 = None
        self.actions = ['check','raise','fold']
        self.cards = {'king':3, 'queen':2, 'jack':1}
        
    def deal(self):
        cards = list(self.cards.keys())
        p1 = random.choice(cards)
        cards.remove(p1)
        p2 = random.choice(cards)
        return p1, p2
    
    def place_bet(self, player, amount):
        self.pot += amount
        setattr(self, player, getattr(self, player) - amount)
    
    def choose_action(self, player, action, amount = 1):
        if isinstance(action, int):
            setattr(self, f'a{player[-1]}', self.actions[action])
        elif str(action).lower() in self.actions:
            setattr(self, f'a{player[-1]}', str(action).lower())
        else:
            return False
        if getattr(self, f'a{player[-1]}') == 'raise' or getattr(self, f'a{player[-1]}') == 'call':
            game.place_bet(player, amount)
        return True
    
    def pay_out(self, p1, p2):
        if self.cards[p1] > self.cards[p2]:
            self.p1 += self.pot
        else:
            self.p2 += self.pot
        self.pot = 0
        
game = Kuhn_Poker()

#Anti up
game.place_bet('p1', 1)
game.place_bet('p2', 1)

#Deal cards
p1, p2 = game.deal()

#Choose action
game.choose_action('p1', 'Check')
game.choose_action('p2', 'Raise')
game.choose_action('p1', 'Call')

game.pay_out(p1, p2)
print(f'player 1 = ${game.p1}\nplayer 2 = ${game.p2}')

player 1 = $4
player 2 = $0


## Nash Equilibrium (Aproximation)
$u_{1}(\sigma) + \epsilon \geq \underset{\sigma^{'}_{1}\epsilon \sum_{1}}{\text{max}}u_{1}(\sigma^{'}_{1},\sigma_{2})$ <br>
$u_{2}(\sigma) + \epsilon \geq \underset{\sigma^{'}_{2}\epsilon \sum_{2}}{\text{max}}u_{2}(\sigma^{'}_{2},\sigma_{2})$

$u=\text{utility function}$ <br>
$\sigma=\text{player strategy}$

What this means is that both players are playing to win, meaning they are each playing their best reponces to the players strategy. Because both players are trying to win this makes the best each player can do is tie. Nash equilibrium has an exploitability of zero meaning it is theoretically a unbeatable strategy making it optimal for most games.

## Regret-Matching+
$r(s,a)=v_{i}(s,a)-\sum_{b \epsilon A(s)}{p(s,b)v_{i}(s,b)}$

$r=\text{beliefs (range)}$ <br>
$v=\text{counterfactual value}$ <br>
$p=\text{player policy}$ <br>
$s=\text{public state}$

$Q^{t}(s,a)=\left(Q^{t-1}(s,a)+r^{t}(s,a)\right)^{+}$

$Q=\text{regret like value}$ <br>
$s=\text{public state}$ <br>
$a=\text{action}$ <br>
$r=\text{beliefs (range)}$ <br>
$t=\text{time step}$

$\pi^{t+1}(s,a)=\frac{Q^{t}(s,a)}{\sum_{b}{Q^{t}(s,b)}}$

$\pi=\text{player strategy}$ <br>
$s=\text{public state}$ <br>
$Q=\text{regret like value}$

## Linearly weighted policy averaging

$\bar{\pi}^{T}_{i}=\frac{2}{(T^{2}+T)\sum^{T}_{t=1}{t\pi^{t}_{i}}}$

$\pi=\text{player strategy}$ <br>
$i=\text{player}$ <br>
$T=\text{Iterations}$

In [269]:
from copy import deepcopy

class GT_CFR:
    def __init__(self, nn, s, c, action_space, k = 1):
        self.T = 0
        self.k = k #Total actions to expand
        self.s = s #Total expansion simulations
        self.c = c #Expansion simulations per regret update
        self.H = [] #History
        self.L = {} #Tree
        self.action_space = action_space
        self.CVPN = nn
        
    class Node:
        def __init__(self):
            self.N = 0
            self.Q = []
            self.v = []
            self.p = []
            self.r = []
            self.CFR_pi = []
            self.pUCT_pi = []
            self.actions = []
        
    def state_hash(self, s):
        result = hash(str(s))
        return result
    
    def CFR(self, s, Q_1 = 0):
        s_hash = self.state_hash(s)
        if len(self.L[s_hash].actions) == 0: 
            B = (s, self.L[s_hash].r)
            v, p = self.CVPN.predict(B)
            self.L[s_hash].v = v
            self.L[s_hash].p = p

        r_sum = sum([v * p for v, p in zip(self.L[s_hash].v, self.L[s_hash].p)])
        for i, v in enumerate(self.L[s_hash].v):
            r = v - r_sum
            self.L[s_hash].r.append(r)
            if len(self.L[s_hash].Q) == len(self.L[s_hash].v):
                self.L[s_hash].Q[i] = max(Q_1 + r, 0)
            else:
                self.L[s_hash].Q.append(max(Q_1 + r, 0))
            
        if len(self.L[s_hash].actions) > 0:
            self.regret_matching_plus(s_hash)
            for i, Q_1 in enumerate(self.L[s_hash].Q):
                self.CFR(self.L[s_hash].actions[i], Q_1 = Q_1)
                
    def regret_matching_plus(self, s_hash):
        #self.L[s_hash].CFR_pi = []
        Q_sum = sum(self.L[s_hash].Q)
        for i, Q in enumerate(self.L[s_hash].Q):
            if Q_sum > 0:
                self.L[s_hash].p[i] = Q / Q_sum
            else:
                self.L[s_hash].p[i] = 0
            '''
            if Q_sum > 0:
                self.L[s_hash].CFR_pi.append(Q / Q_sum)
            else:
                self.L[s_hash].CFR_pi.append(0)
            '''
    
    def expansion(self, s):
        s_hash = self.state_hash(s)
        if len(self.L[s_hash].actions) > 0:
            self.regret_matching_plus(s_hash)
            #self.pUCT(s_hash)
            pi_bank = []
            for rm_plus in self.L[s_hash].p:
            #for rm_plus, pUCT in zip(self.L[s_hash].p, self.L[s_hash].pUCT_pi):
                pi = rm_plus
                #pi = (.5 * rm_plus) + (.5 * pUCT)
                pi_bank.append(pi)
            m_pi = max(pi_bank)
            a_bank = [i for i, v in enumerate(pi_bank) if v == m_pi]
            a = random.choice(a_bank)
            self.expansion(self.L[s_hash].actions[a])
        else:
            #ADD ACTIONS TO TREE
            for a in range(self.action_space):
                a_hold = deepcopy(s)
                if a == 1 and a_hold[0] == 1:
                    a_hold[-2] += 1
                elif a == 2 and a_hold[0] == 1:
                    a_hold[-1] += a_hold[-2]
                    a_hold[-2] = 0
                elif (a == 0 or a == 1) and a_hold[0] == -1:
                    a_hold[2] = 3 if a_hold[1] == 2 else 2
                elif a == 1:
                    a_hold[-1] += 1
                elif a == 2:
                    a_hold[-2] += a_hold[-1]
                    a_hold[-1] = 0
                a_hold[0] *= -1
                search.L[s_hash].actions.append(a_hold)
                a_hash = self.state_hash(a_hold)
                search.L[a_hash] = search.Node()
        self.L[s_hash].N += 1 #Update node visit count
    
    def GT_CFR(self, I):
        for _ in range(int(self.s / self.c)):
            self.CFR(I) #Run CFR
            for _ in range(self.c):
                self.expansion(I) #Grow tree

class CVPN:
    def predict(self, B):
        s, r = B
        c = max(s)
        if c == 3:
            v = [0.5, 0.5, 0]
            p = [0.25, 0.25, 0.25]
        elif c == 2:
            v = [0.25, 0.25, 0.25]
            p = [0.25, 0.25, 0.25]
        else:
            v = [0.5, 0, 0.5]
            p = [0.25, 0.25, 0.25]
        return v, p

In [270]:
nn = CVPN()
search = GT_CFR(nn, 2, 1, 3)

game = Kuhn_Poker()

#Anti up
game.place_bet('p1', 1)
game.place_bet('p2', 1)

#Deal cards
p1, p2 = game.deal()
p1 = 'king'
p2 = 'queen'

print(p1, p2)
s = [1] + [game.cards[p1]] + [-1, 1, 1]
s_hash = search.state_hash(s)
search.L[s_hash] = search.Node()
search.L[s_hash].r = [1/(len(game.cards) - 1) if c != p1 else 0 for c in game.cards]

search.GT_CFR(s)

print(search.L[s_hash].actions)
print(search.L[s_hash].Q)
'''
#Choose action
game.choose_action('p1', 'Check')
game.choose_action('p2', 'Raise')
game.choose_action('p1', 'Call')

game.pay_out(p1, p2)
print(f'player 1 = ${game.p1}\nplayer 2 = ${game.p2}')
'''

king queen
[[-1, 3, -1, 1, 1], [-1, 3, -1, 2, 1], [-1, 3, -1, 0, 2]]
[0.25, 0.25, 0]


"\n#Choose action\ngame.choose_action('p1', 'Check')\ngame.choose_action('p2', 'Raise')\ngame.choose_action('p1', 'Call')\n\ngame.pay_out(p1, p2)\nprint(f'player 1 = ${game.p1}\nplayer 2 = ${game.p2}')\n"

## Average overall Regret
$R^{T}_{i}=\frac{1}{T} \underset{\sigma^{*}_{i}\epsilon \sum_{i}}{\text{max}} \sum^{T}_{t=1}\left(u_{i}(\sigma^{*}_{i},\sigma^{t}_{-i})-u_{i}(\sigma^{t})\right)$

$i=\text{player}$ <br>
$\sigma=\text{player strategy}$ <br>
$u=\text{utitlity function}$ <br>
$T=\text{current time}$

## Regret Matching
$v_{i}(\sigma,h)=\sum_{z \epsilon Z, h \sqsubset z} \pi^{\sigma}_{-i}(h)\pi^{\sigma}_{i}(h,z)u_{i}(z)$

$\pi=\text{policy at curren time step}$ <br>
$h=\text{current state}$ <br>
$u = \text{utility function at current time step for action}$ <br>
$i = \text{player (negative i refers to the opponent)}$

<br>
<br>

$\Delta R^{T}(a)=v^t(a)-\sum_{b \epsilon A}{\sigma^{t}(b)v^{t}(b)}$

$v=\text{counterfactual value}$ <br>
$\sigma = \text{player policy}$ <br>
$t=\text{timestep}$

<br>
<br>

$R^{t}(a)=R^{t-1}(a)+\Delta R^{t}(a)$

$R=\text{regret}$ <br>
$a=\text{action}$ <br>
$t=\text{time step}$

<br>
<br>

$\sigma^{t}(a)=R^{t-1}(a)^{+}/\sum_{b \epsilon A}{R^{t-1}(b)^{+}}$

$R=\text{regret}$ <br>
$t=\text{time step}$ <br>
$x^{+}=max(x,0)$

<br>
<br>

$u^{\sigma}_{i}=\sum_{z}\pi^{\sigma}(z)$

In [7]:
# kuhn poker time
# 3 cards
# 3*2 = 6 hands

HANDS = [(1,2), (1,3), (2,1), (2,3), (3,1), (3,2)]

# there are 12 information sets
# p1(1st) 1 {2,3}
# p1(1st) 2 {1,3}
# p1(1st) 3 {1,2}
# p2(2nd) 1, p1 pass {2,3}
# p2(2nd) 1, p1 bet  {2,3}
# p2(2nd) 2, p1 pass {1,3}
# p2(2nd) 2, p1 bet  {1,3}
# p2(2nd) 3, p1 pass {1,2}
# p2(2nd) 3, p1 bet  {1,2}
# p1(3rd) 1, p1 pass, p2 bet  {2,3}
# p1(3rd) 2, p1 pass, p2 bet  {1,3}
# p1(3rd) 3, p1 pass, p2 bet  {1,2}

ISETS = ["1", "2", "3",   # round 1
         "P1", "P2", "P3", "B1", "B2", "B3",  # round 2
         "PB1", "PB2", "PB3"]  # round 3

# terminal history states
TERMINAL = ["PP", "PBP", "PBB", "BP", "BB"]
ACTIONS = ["P", "B"]

def payout(rs, h):
    if h == "PBP":
        return -1
    elif h == "BP":
        return 1
    m = 1 if (rs[0] > rs[1]) else -1
    if h == "PP":
        return m
    if h in ["BB", "PBB"]:
        return m*2
    assert False

def get_information_set(rs, h):
    assert h not in TERMINAL
    if h == "":
        return str(rs[0])
    elif len(h) == 1:
        return h + str(rs[1])
    else:
        return "PB" + str(rs[0])
    assert False

def cfr(rs, h, i, t, pi1, pi2):
    # rs = realstate
    # h = history
    # i = player
    # t = timestep
    # pi1 = reach probability player 1
    # pi2 = reach probabilty player 2
    
    if h in TERMINAL:
        return payout(rs, h) * (1 if i == 1 else -1)
    I = get_information_set(rs, h)
    ph = 2 if len(h) == 1 else 1 #Player hand
    
    # if we are here, we have both actions available
    vo = 0.0 #Counter factual value
    voa = {}
    for a in ACTIONS:
        if ph == 1: #Player 1
            voa[a] = cfr(rs, h+a, i, t, sigma[t][I][a] * pi1, pi2)
        else: #Player 2
            voa[a] = cfr(rs, h+a, i, t, pi1, sigma[t][I][a] * pi2)
        vo += sigma[t][I][a] * voa[a]
    if ph == i:
        if i == 1:
            pi = pi1
            pnegi = pi2
        else:
            pi = pi2
            pnegi = pi1
        for a in ACTIONS:
            regret[I][a] += pnegi * (voa[a] - vo)
            strategy[I][a] += pi * sigma[t][I][a] #Since PI is set @ 1 the strategy is set to current sigma
        # update the strategy based on regret
        rsum = sum([max(x, 0) for x in regret[I].values()])
        for a in ACTIONS:
            if rsum > 0:
                sigma[t+1][I][a] = max(regret[I][a], 0) / rsum
            else:
                sigma[t+1][I][a] = 0.5
    return vo

In [8]:
# init tables
regret = {}
strategy = {}
for I in ISETS:
    regret[I] = {k:0 for k in ACTIONS}
    strategy[I] = {k:0 for k in ACTIONS}
    
sigma = {}
sigma[1] = {}
for I in ISETS:
    sigma[1][I] = {k:0.5 for k in ACTIONS}

# learn strategy
import copy
import random
for t in range(1, 1000):
    sigma[t+1] = copy.deepcopy(sigma[t])
    for i in [1,2]:
        cfr(random.choice(HANDS), "", i, t, 1, 1)
    del sigma[t]

In [9]:
# print "average" strategy
for k,v in strategy.items():
    norm = sum(list(v.values()))
    print("%3s: P:%.4f B:%.4f" % (k, v['P']/norm, v['B']/norm))
# https://en.wikipedia.org/wiki/Kuhn_poker#Optimal_strategy

  1: P:0.7530 B:0.2470
  2: P:0.9665 B:0.0335
  3: P:0.3649 B:0.6351
 P1: P:0.5789 B:0.4211
 P2: P:0.9941 B:0.0059
 P3: P:0.0174 B:0.9826
 B1: P:0.9984 B:0.0016
 B2: P:0.5653 B:0.4347
 B3: P:0.0015 B:0.9985
PB1: P:0.9991 B:0.0009
PB2: P:0.5009 B:0.4991
PB3: P:0.0023 B:0.9977
